In [ ]:
import pickle

import pandas as pd
import weave
from IPython.display import display, Markdown

from ydnpd.generation.agent.specifications import *
from ydnpd.generation.config import LLM_PATHS
from ydnpd.harness import ALL_EXPERIMENTS
from ydnpd.generation.csv import generate_csv

weave.init("ydnpd-data_gen_csv")

In [2]:
PRIVATE_DATASET = "acs/national"

In [ ]:
results = []

for llm_path in LLM_PATHS:
    
    display(Markdown(f"## {llm_path}"))
    
    dataset = generate_csv(
        PRIVATE_DATASET,
        verbose=True
    )        
    results.append({"llm_path": llm_path,
                    "dataset": dataset})

In [ ]:
results

In [ ]:
# with open("llm-datasets.pkl", "wb") as f:
#     pickle.dump(results, f)

# with open("llm-datasets.pkl", "rb") as f:
#     results = pickle.load(f)

In [ ]:
llm_datasets = {}

llm_gorupby = pd.DataFrame(results).groupby(["llm_name", "llm_temperature", "specificaion_version"])

for key, group in llm_gorupby:
    name = "-".join(map(str, key))

    display(Markdown(f"## {name}"))

    group_llm_datasets = {}

    for i, (_, row) in enumerate(group.iterrows()):
        group_llm_datasets[f"{name}-{i}"] = row["dataset"]
    
    fig, _ = plot_dataset_selection(list(group_llm_datasets.values()))
    display(fig)

    llm_datasets |= group_llm_datasets

plot_dataset_selection(list(llm_datasets.values()));

In [6]:
MIXTURE_MAX_COV_OVERALL_NUM_DATASETS = 10
MIXTURE_MAX_COV_PER_GROUP_NUM_DATASETS = 5

mixture_llm_datasets = {
    "MIX-UNIF": mix_datasets(list(llm_datasets.values()), NUM_SAMPLES, "uniform"),
    "MIX-MAX": mix_datasets(list(llm_datasets.values()), NUM_SAMPLES, "max_coverage", MIXTURE_MAX_COV_OVERALL_NUM_DATASETS)

}

for key, group in llm_gorupby:
    name = "-".join(map(str, key))

    mixture_llm_datasets[f"{name}-MIX-UNIF"] = mix_datasets(list(group["dataset"]), NUM_SAMPLES, "uniform")
    mixture_llm_datasets[f"{name}-MIX-MAX"] = mix_datasets(list(group["dataset"]), NUM_SAMPLES, "max_coverage", MIXTURE_MAX_COV_PER_GROUP_NUM_DATASETS)


llm_datasets |= mixture_llm_datasets

In [ ]:
datasets = {
    name.split("/")[1]: load_dataset(name)[0]
    for name in ALL_EXPERIMENTS[FAMILY].dev_names
} | llm_datasets

plot_distribution_distances(datasets)

In [10]:
for name, df in datasets.items():
    if "MIX" in name:
        df.to_csv(f"llm_datasets/{FAMILY}/{name.replace('.','_')}.csv")